In [2]:
import javalang
import time

In [3]:
ast = javalang.parse.parse("public class A { public void m() { int i = 0; } }")

In [13]:
with open("./Snippets/BogoSort.java") as f:
    # read the file
    code = f.read()
    # generate ast from code
    ast = javalang.parse.parse(code)
    for path, node in ast.filter(javalang.tree.TypeDeclaration):
        for node in node.methods:
            print(node.name)

main
bogo
isFinished


In [6]:
from javalang.tree import CompilationUnit

help(CompilationUnit)

Help on class CompilationUnit in module javalang.tree:

class CompilationUnit(javalang.ast.Node)
 |  CompilationUnit(**kwargs)
 |  
 |  Method resolution order:
 |      CompilationUnit
 |      javalang.ast.Node
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  attrs = ['package', 'imports', 'types']
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from javalang.ast.Node:
 |  
 |  __equals__(self, other)
 |  
 |  __init__(self, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  filter(self, pattern)
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties inherited from javalang.ast.Node:
 |  
 |  children
 |  
 |  position
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from javalan